In [1]:
import json
import tensorflow as tf
import numpy as np
from typing import Optional
from src.models.ig_cnn import ig_cnn_model


In [2]:
def gen_conv(filters:int, kernel_size:int, activation:Optional[str]=None, padding:Optional[str]=None):
   _conv = f"C-{filters}-{kernel_size}"
   if activation is not None:
       _conv += f"-{activation}"
   if padding is not None:
       _conv += f"-{padding}"
   return _conv

def gen_dense(units: int, activation: Optional[str]=None):
    _dense = f"D-{units}"
    if  activation is not None:
       _dense += f"-{activation}"
    return _dense

### Generate CNN configs

In [3]:
idx = 32
for phantom_size in [7]: #[3, 5]: # 7]:
    print(f"Input {phantom_size}")
    for activation in ['sigmoid', 'relu']:
        for filters in [10, 15, 25, 50]:
            for kernel in [3, 5, 7]:
                i = phantom_size
                depth = 0
                layers = ""
                while i >= kernel:
                    depth += 1
                    i -= (kernel-np.floor(kernel/2.0))
                    layers += f"{gen_conv(filters, kernel, activation, padding='valid')},"
                    # Test if model is creatable
                    model_def = f"{layers}F"
                    model = ig_cnn_model(model_def, input_size=(phantom_size, phantom_size))
                    model.compile(loss='mse')
                    with open(f"{idx}.json", 'w') as file:
                        config = dict(model_def=model_def,
                                      activation=activation,
                                      kernel=kernel,
                                      filters=filters,
                                      depth=depth,
                                      phantom_size=phantom_size,
                                      log_name=f'{phantom_size}/{model_def}/',
                                      log_root='../logs/',
                                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                                 )
                        json.dump(config, file)
                    idx += 1

Input 7


### Generate Baseline configs

In [31]:
idx = 0
for phantom_size in [3, 5]:
    # Test if model is creatable
    model_def = f'dense_baseline_{phantom_size}'
    with open(f"baseline_{phantom_size}.json", 'w') as file:
        config = dict(model_def=model_def,
                      phantom_size=phantom_size,
                      log_name=f'{phantom_size}/baseline/',
                      log_root='../logs/',
                      phantom_root=f'../data/phantoms/phantom_{phantom_size}_{phantom_size}/training',
                      phantom_file_wildcard=f'*_phantom_{phantom_size}_{phantom_size}.npz'
                 )
        json.dump(config, file)
    idx += 1

Results

In [4]:
import pandas as pd
import plotly.express as px

In [5]:
df = pd.concat([pd.read_csv('conv5.csv'), pd.read_csv('conv3.csv')])

,log_name,phantom_size,model_def,phantom_file,log_root,phantom_root,phantom_sample,phantom_file_wildcard,activation,filters,depth,kernel,Information Gain train,Information Gain train.1,Information Gain valid,Information Gain valid.1,epoch_loss,epoch_loss.1,epoch_mean_squared_error,epoch_mean_squared_error.1
0,"5/C-10-3-relu-valid,F/",5,"C-10-3-relu-valid,F",../data/phantoms/phantom_5_5/training/6_phanto...,../logs/,../data/phantoms/phantom_5_5/training,6,*_phantom_5_5.npz,relu,10.0,1.0,3.0,0.342517,0.019421,0.391837,0.019480,0.000569,0.000653,0.000569,0.000653
1,"5/C-50-3-relu-valid,C-50-3-relu-valid,F/",5,"C-50-3-relu-valid,C-50-3-relu-valid,F",../data/phantoms/phantom_5_5/training/1_phanto...,../logs/,../data/phantoms/phantom_5_5/training,1,*_phantom_5_5.npz,relu,50.0,2.0,3.0,0.313694,0.018111,0.276933,0.018142,0.000442,0.000574,0.000442,0.000574
2,"5/C-15-3-sigmoid-valid,C-15-3-sigmoid-valid,F/",5,"C-15-3-sigmoid-valid,C-15-3-sigmoid-valid,F",../data/phantoms/phantom_5_5/training/4_phanto...,../logs/,../data/phantoms/phantom_5_5/training,4,*_phantom_5_5.npz,sigmoid,15.0,2.0,3.0,0.284498,0.014458,0.300218,0.014542,0.000384,0.000363,0.000384,0.000363
3,5/baseline/,5,dense_baseline_5,../data/phantoms/phantom_5_5/training/19_phant...,../logs/,../data/phantoms/phantom_5_5/training,19,*_phantom_5_5.npz,NaN,NaN,NaN,NaN,0.229255,0.015672,0.218341,0.015798,0.000431,0.000419,0.000431,0.000419
4,"5/C-25-3-relu-valid,F/",5,"C-25-3-relu-valid,F",../data/phantoms/phantom_5_5/training/6_phanto...,../logs/,../data/phantoms/phantom_5_5/training,6,*_phantom_5_5.npz,relu,25.0,1.0,3.0,0.345030,0.016259,0.374681,0.016241,0.000507,0.000482,0.000507,0.000482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,"3/C-10-3-relu-valid,F/",3,"C-10-3-relu-valid,F",../data/phantoms/phantom_3_3/training/6_phanto...,../logs/,../data/phantoms/phantom_3_3/training,6,*_phantom_3_3.npz,relu,10.0,1.0,3.0,0.277928,0.016979,0.281130,0.017187,0.000528,0.000545,0.000528,0.000545
176,"3/C-10-3-relu-valid,F/",3,"C-10-3-relu-valid,F",../data/phantoms/phantom_3_3/training/3_phanto...,../logs/,../data/phantoms/phantom_3_3/training,3,*_phantom_3_3.npz,relu,10.0,1.0,3.0,0.178136,0.016365,0.140548,0.016436,0.000465,0.000434,0.000465,0.000434
177,3/baseline/,3,dense_baseline_3,../data/phantoms/phantom_3_3/training/16_phant...,../logs/,../data/phantoms/phantom_3_3/training,16,*_phantom_3_3.npz,NaN,NaN,NaN,NaN,0.398881,0.018042,0.469113,0.017994,0.000568,0.000615,0.000568,0.000615
178,3/baseline/,3,dense_baseline_3,../data/phantoms/phantom_3_3/training/7_phanto...,../logs/,../data/phantoms/phantom_3_3/training,7,*_phantom_3_3.npz,NaN,NaN,NaN,NaN,0.470989,0.020315,0.319381,0.020329,0.000851,0.000824,0.000851,0.000824


In [6]:
df.groupby(['model_def', 'phantom_size'])
df = df.rename(columns={'Information Gain valid': 'ig_max_valid',
                        'Information Gain valid.1': 'ig_mean_valid',
                        'Information Gain train': 'ig_max_train',
                        'Information Gain train.1': 'ig_mean_train',
                        'epoch_loss.1': 'mse_valid'})

In [7]:
df.columns

Index(['log_name', 'phantom_size', 'model_def', 'phantom_file', 'log_root',
       'phantom_root', 'phantom_sample', 'phantom_file_wildcard', 'activation',
       'filters', 'depth', 'kernel', 'ig_max_train', 'ig_mean_train',
       'ig_max_valid', 'ig_mean_valid', 'epoch_loss', 'mse_valid',
       'epoch_mean_squared_error', 'epoch_mean_squared_error.1'],
      dtype='object')

In [28]:
df = df.astype({'phantom_size': 'int32',
                'kernel': 'int32',
                'phantom_sample': 'int32',
                'depth': 'int32',
                'filters': 'int32'})

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [8]:
(
    df
        .drop(columns=['log_root',
                       'phantom_file_wildcard',
                       'phantom_root',
                       'log_name',
                       'epoch_mean_squared_error',
                       'epoch_mean_squared_error.1',
                       'phantom_sample'])
        .groupby(['model_def', 'phantom_size'])
        .mean()
).sort_values('mse_valid')

,,filters,depth,kernel,ig_max_train,ig_mean_train,ig_max_valid,ig_mean_valid,epoch_loss,mse_valid
model_def,phantom_size,,,,,,,,,
"C-25-3-sigmoid-valid,C-25-3-sigmoid-valid,F",5,25.0,2.0,3.0,0.267442,0.015489,0.255679,0.015515,0.000423,0.000427
"C-15-3-sigmoid-valid,C-15-3-sigmoid-valid,F",5,15.0,2.0,3.0,0.269380,0.015952,0.257479,0.015967,0.000445,0.000450
"C-50-3-sigmoid-valid,C-50-3-sigmoid-valid,F",5,50.0,2.0,3.0,0.268087,0.016108,0.247248,0.016126,0.000428,0.000454
"C-50-3-relu-valid,C-50-3-relu-valid,F",5,50.0,2.0,3.0,0.278515,0.016139,0.249773,0.016182,0.000459,0.000465
"C-15-3-relu-valid,C-15-3-relu-valid,F",5,15.0,2.0,3.0,0.264283,0.016418,0.252701,0.016446,0.000473,0.000475
"C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,F",5,10.0,2.0,3.0,0.288986,0.016384,0.273372,0.016408,0.000476,0.000486
"C-25-3-relu-valid,C-25-3-relu-valid,F",5,25.0,2.0,3.0,0.283577,0.016401,0.263333,0.016457,0.000468,0.000488
"C-10-3-relu-valid,C-10-3-relu-valid,F",5,10.0,2.0,3.0,0.281760,0.016815,0.262950,0.016837,0.000492,0.000509
"C-25-5-sigmoid-valid,F",5,25.0,1.0,5.0,0.281613,0.016906,0.270178,0.016966,0.000503,0.000526


In [25]:
df.groupby

,kernel,phantom_sample,model_def,filters,depth,phantom_size,activation,ig_max_train,ig_mean_train,ig_max_valid,ig_mean_valid,epoch_loss,mse_valid,epoch_mean_squared_error,epoch_mean_squared_error.1
0,3,4,"C-15-3-relu-valid,F",15,1,3,relu,0.331959,0.014857,0.273843,0.014797,0.000462,0.000422,0.000462,0.000422
1,3,6,"C-25-3-relu-valid,F",25,1,3,relu,0.256417,0.017329,0.235471,0.017470,0.000444,0.000513,0.000444,0.000513
2,3,4,"C-25-3-relu-valid,F",25,1,3,relu,0.343432,0.021618,0.283914,0.021567,0.000522,0.000766,0.000522,0.000766
3,3,2,"C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,F",10,2,5,sigmoid,0.338693,0.017929,0.375785,0.017777,0.000620,0.000587,0.000620,0.000587
4,5,5,"C-50-5-relu-valid,F",50,1,5,relu,0.272704,0.017764,0.246749,0.017821,0.000507,0.000552,0.000507,0.000552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,3,9,"C-25-3-relu-valid,F",25,1,5,relu,0.162755,0.014911,0.140608,0.014974,0.000381,0.000365,0.000381,0.000365
316,3,9,"C-25-3-relu-valid,F",25,1,3,relu,0.465206,0.019345,0.339105,0.019356,0.000729,0.000702,0.000729,0.000702
317,3,9,"C-15-3-relu-valid,C-15-3-relu-valid,F",15,2,5,relu,0.181405,0.018352,0.161519,0.018453,0.000377,0.000550,0.000377,0.000550
318,5,9,"C-25-5-relu-valid,F",25,1,5,relu,0.161423,0.015291,0.149667,0.015401,0.000426,0.000391,0.000426,0.000391


In [ ]:
px.box(df, x='model_def', y='mse_valid', color='phantom_size')

In [14]:
df[['kernel', 'phantom_sample', ]]

,kernel,phantom_sample,phantom_file_wildcard,model_def,filters,depth,phantom_size,log_root,phantom_root,activation,log_name,Information Gain train,Information Gain train.1,ig_max_valid,ig_mean_valid,epoch_loss,mse_valid,epoch_mean_squared_error,epoch_mean_squared_error.1
0,3.0,4.0,*_phantom_3_3.npz,"C-15-3-relu-valid,F",15.0,1.0,3.0,../logs/,../data/phantoms/phantom_3_3/training,relu,"3/C-15-3-relu-valid,F/",0.331959,0.014857,0.273843,0.014797,0.000462,0.000422,0.000462,0.000422
1,3.0,6.0,*_phantom_3_3.npz,"C-25-3-relu-valid,F",25.0,1.0,3.0,../logs/,../data/phantoms/phantom_3_3/training,relu,"3/C-25-3-relu-valid,F/",0.256417,0.017329,0.235471,0.017470,0.000444,0.000513,0.000444,0.000513
2,3.0,4.0,*_phantom_3_3.npz,"C-25-3-relu-valid,F",25.0,1.0,3.0,../logs/,../data/phantoms/phantom_3_3/training,relu,"3/C-25-3-relu-valid,F/",0.343432,0.021618,0.283914,0.021567,0.000522,0.000766,0.000522,0.000766
3,3.0,2.0,*_phantom_5_5.npz,"C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,F",10.0,2.0,5.0,../logs/,../data/phantoms/phantom_5_5/training,sigmoid,"5/C-10-3-sigmoid-valid,C-10-3-sigmoid-valid,F/",0.338693,0.017929,0.375785,0.017777,0.000620,0.000587,0.000620,0.000587
4,5.0,5.0,*_phantom_5_5.npz,"C-50-5-relu-valid,F",50.0,1.0,5.0,../logs/,../data/phantoms/phantom_5_5/training,relu,"5/C-50-5-relu-valid,F/",0.272704,0.017764,0.246749,0.017821,0.000507,0.000552,0.000507,0.000552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,3.0,9.0,*_phantom_5_5.npz,"C-25-3-relu-valid,F",25.0,1.0,5.0,../logs/,../data/phantoms/phantom_5_5/training,relu,"5/C-25-3-relu-valid,F/",0.162755,0.014911,0.140608,0.014974,0.000381,0.000365,0.000381,0.000365
316,3.0,9.0,*_phantom_3_3.npz,"C-25-3-relu-valid,F",25.0,1.0,3.0,../logs/,../data/phantoms/phantom_3_3/training,relu,"3/C-25-3-relu-valid,F/",0.465206,0.019345,0.339105,0.019356,0.000729,0.000702,0.000729,0.000702
317,3.0,9.0,*_phantom_5_5.npz,"C-15-3-relu-valid,C-15-3-relu-valid,F",15.0,2.0,5.0,../logs/,../data/phantoms/phantom_5_5/training,relu,"5/C-15-3-relu-valid,C-15-3-relu-valid,F/",0.181405,0.018352,0.161519,0.018453,0.000377,0.000550,0.000377,0.000550
318,5.0,9.0,*_phantom_5_5.npz,"C-25-5-relu-valid,F",25.0,1.0,5.0,../logs/,../data/phantoms/phantom_5_5/training,relu,"5/C-25-5-relu-valid,F/",0.161423,0.015291,0.149667,0.015401,0.000426,0.000391,0.000426,0.000391


In [15]:
df['phantom_size']

0      3.0
1      3.0
2      3.0
3      5.0
4      5.0
      ... 
315    5.0
316    3.0
317    5.0
318    5.0
319    5.0
Name: phantom_size, Length: 320, dtype: float64